# Activity report

Jérémie Decock, 2017-06-12

## Context

CTA will be a big step forward for gamma ray science.
Its sensitivity will be 10 times better than existing (e.g. H.E.S.S.) but we want to do even better using advanced signal processing methods to improve the signal-to-noise ratio at the level of the instrument (removing the electronic instrument's noise and the night sky background before shower reconstruction).

#### Goal

My work consists of implementing an advanced image cleaning methods based on Wavelet Transforms using tools developed by the Cosmostat team to improve the signal-to-noise ratio of cleaned images (at the level of the instrument) and thus improve the accuracy of the shower reconstruction and the sensibility of the observatory.

These Cosmostat tools are part of the ISAP/Sparse2d library.
Among the available modules, I mostly use the *mr_transform* and *mr_filter*.

#### Data set

So far, I compared image cleaning algorithms using Monte-Carlo simulations of air showers.
These air showers are observed with ASTRI and GCT telescopes in the simulations.

#### ASTRI

Most of my work has been done on ASTRI mini array simulations containing 33 ASTRI telescopes.

I used a sub set of ~15000 images for gamma and for proton simulations (~100 GeV to ~300 TeV).

#### GCT

Recently I also worked on GCT simulations (31 telescopes).

I used a sub set of ~20000 images for proton simulations.
I hasn't worked on gamma simulation yet.

#### Cleaning algorithm reference

The "Tailcut clean" algorithm:
- Keep pixels above a given threshold (10 PE)
- Keep some neighbors of these selected pixels: those above a second (lower) threshold (5 PE)

## What have been done so far: problems encountered, solutions found and solutions abandoned

#### Incorporate Cosmostat image cleaning tools in the CTA data framework

I have written a dedicated Python framework to make the link between the Cosmostat tools and the CTA data framework: https://github.com/jdhp-sap/sap-cta-data-pipeline

It is flexible enough to be easily implemented in ctapipe as an external optional tool.

#### Fetch data to assess our image cleaning framework

We first made ourselves our simulated data set using Corsika/SimtelArray.

Then we switched to the ASTRI mini array data set. We found calibration bugs in ctapipe/pyhessio with these data that slowed us down for several weeks.

I now use GCT and ASTRI data sets and I'm going to switch to Prod3 data.

#### Use the right Cosmostat tool

Cosmostat's ISAP/sparse2d library provides several tools for image cleaning.
I first used mr_transform which only make the wavelet transform, without any filtering or final image reconstruction.

I then used mr_filter that does the full image cleaning process (wavelet transform, filtering and reconstruction).

Both tools have pros and cons:
- mr_transform is much more flexible (this may be necessary in a near future if we implement some models specific to our data)
- mr_transform have a Python binding ready to be used (mr_filter will have one in autumn 2017)
- mr_filter offer much more options ready for use

#### Find the right estimator to assess image cleaning

The final estimator of our cleaning method is the sensitivity.
For practical reasons, we also needed lower level estimators to assess our image cleaning methods.

Among the main metrics I use are:
- the relative total counts difference between the clean and the cleaned image (to assess the loss in energy)
- the error of the shower axis reconstruction on the image
    - that is to say the difference between the angle of the shower computed from the clean image and from the cleaned image
    - this estimator is interesting as it gives a reasonably good indication on the future performances of event reconstruction with stereoscopy

Many other estimators have been tested, the full list is available here: http://www.jdhp.org/software/sap-cta-data-pipeline/api_benchmark_assess.html

#### Find the right set of options for mr_filter

Following advises from the Cosmostat team (mainly from Sandrine, Florent and Jean-Luc), I tested many options.

Some options have been optimized automatically (e.g. the *sigma thresholds* that define coefficients to keep for each plane is a regular optimization problem in $\mathbb{R}^4$).
The others have been tested empirically on some representative chozen examples with a dedicated GUI before being assessed on the full data set.

##### b-spline wavelets "a trous"

After several optimizations on a subset of available methods and parameters, and following some advices from the Cosmostat team, I initially used a configuration based on b-spline wavelets "a trous".

The mr_filter setting corresponding to this configuration (*Hard K-Sigma Thresholding using Poisson + Gaussian noise model*) is:

    ./datapipe/denoising/wavelets_mrfilter.py -K -k -C1 -m3 -n4 -s3 --kill-isolated-pixels

I looked performance on 2 populations: faint showers (50 to 100 NPE) and bright showers (100 to 2000 NPE).

- For faint showers, wavelets have more images than Tailcut with a precise reconstruction angle of showers and less than Tailcut with a bad reconstruction angle.
- For brighter showers, Tailcut does better

This was not dramatic as what we want in priority is to improve performances on faint showers as this is where we can expect an improvement of the sensibility and as Tailcut already perform very well for bright ones (we can imagine to use Wavelets only for faint images).

But with this configuration, there is a shift of the wavelet cleaned population to lower energy and thus a loss of information with this wavelet configuration, which is annoying...

##### Anisotropic wavelets

I followed several suggestions from the Cosmostat team to solve this issue.

The first one is to anisotropic wavelets as our signal is anisotropic and using isotropic wavelets has tendencies to make the signal isotropic and thus to suppress information about the orientation of the shower.

This configuration indeed fix the charge loss.
But so far, it has tendency to make fake substructures and these substructures have a big impact on the accuracy of reconstructed shower axis.

With this configuration wavelet is less accurate than Tailcut.
This can probably be improved playing with other parameters.

##### Iterative Multiresolution Thresholding

In the meanwhile,
I followed another suggestion: use an adaptive threshold estimate on each plane.

This configuration also fix the charge loss.
But again, in the current stage, this configuration has tendency to make fake substructures
that have a negative impact on the shower axis reconstruction.

##### Explicitly reduce the filtering on small scales

An alternative idea to reduce the charge loss and to improve the shower axis reconstruction
was to explicitly reduce the filtering threshold on small scales (first planes).

The motivation is that our signal is mainly contained at small scales.

With this configuration, the energy is well conserved and the shower axis reconstruction is more accurate with Wavelets than Tailcut on both energy ranges.
It does better than the first configuration.

The mr_filter setting corresponding to this configuration (*Hard K-Sigma Thresholding using Poisson + Gaussian noise model*) is:

    ./datapipe/denoising/wavelets_mrfilter.py -K -k -C1 -m3 -n4 -s2,2,3,3 --kill-isolated-pixels
    
##### Other (abandoned) options

Among the other tested and abandoned options are:

- False Discovery Rate

    ./datapipe/denoising/wavelets_mrfilter.py  -K -C2 -m1 -n4 -f2

- Wiener Filtering (min MSE between random process and desired process)
    
    ./datapipe/denoising/wavelets_mrfilter.py  -K -m1 -n4 -f6

##### Wavelet Transforms on a logarithmic images space

Problem: we observe big loss of signal after (wavelets) cleaning for very bright showers

Idea: apply Wavelet Transforms on a logarithmic images space

Status: the idea seems good but it was abandoned as it gives bad results in practice

Actually the issue with bright showers is mostly caused by a saturation of the signal on ASTRI images

TODO: this should be tested again now that we know how to define a different threshold for each plane

#### Use more adapted transformation

Shower images are anisotropic signals.
Using isotropic wavelets has tendencies to make these signals isotropic and thus to suppress information about the orientation of the shower.

Bi-orthogonal wavelets transforms and Curvlet transforms have been tested without success so far.

This path has been (temporary) abandoned.

#### Remove isolated pixels after cleaning

Contrary to Tailcut, many isolated pixels may remain on images cleaned with Wavelets (at least with tested options).
This is annoying as it may disturb a lot Hillas parametrization.

Abandoned solution:
we fought mf_filter's *-k* option would solve this issue but this is not the case.
According to Jean-Luc, the *-k* option:
- only removes completely isolated pixels (i.e. islands with a "size" of 1)
- only act on the first plane (smallest scale)

Solution:
with Fabio, we finally used an external tool (scipy.ndimage) to remove isolated pixels after image cleaning. It improves a lot reconstructed angle showers on the image.

#### Images with empty corner and hexagonal pixel layouts

Cosmostat tools only work with images defined as a rectangular grid of square pixels.

Problem: none of the instruments used in CTA have this kind of pixel layout.
- On some (Astri, GCT), images are defined as a rectangular grid of square pixels but have "empty" parts (in corners) and have various gap size between pixels.
- Others use an hexagonal pixel layout.

In order to assess our image cleaning methods as soon as possible, I first worked on instruments with a square pixel layout (ASTRI), removing some part of the images in order to have data directly compatible with Cosmostat tools.

Now that results are satisfying enough for these images, I still work on instruments with a square pixel layout (ASTRI and GCT) but without removing any part of the images (i.e. using the full available signal).

These empty parts cannot simply be considered as blank signal pixels as it would introduce a lot of harmonics that may have negative impact in the cleaned images (see https://dsm-trac.cea.fr/cta/attachment/wiki/Data/sapcta_weekly_meeting_2017_06_08.pdf).
An *mr_filter* option (``-I``) was expected to solve this problem but actually it only act on the noise model construction.
Alternative solutions currently investigated to solve the "empty parts" issue are:
- Insert artificial noise in the corners using the actual noise distribution
- In-painting: something similar but more sophisticated (as suggested by Florent Sureau)
- Patch the wavelet transform code to anticipate mirroring on NaN pixels

Next step: work on instruments having an hexagonal grid of pixels.

#### Mr_filter wrapper/binding

So far, mr_filter (and mr_transform) read input data and write output data in files.

Problem:
This slow down the full image cleaning process and prevent to fulfill CTA timing constraints.

Long term solution:
A python binding is undergoing form mr_filter. It is coordinated by Cosmostat (but written by DRF/DSV engineers) and should be finished in autumn 2017.
This binding will allow direct data exchange (in RAM) between our framework and mr_filter/mr_transform.
A similar binding has been written for mr_transform.

Temporary solution:
In the meanwhile, I simply read and write data from a ramdisk (a virtual disk volume mounted in RAM).
The speedup (see https://drf-gitlab.cea.fr/CTA-Irfu/Data/wikis/astri-images-cleaning-speed-test):
- x2 if temporary files are put in the ramdisk
- x10 if temporary and input files are put in the ramdisk

Alternative temporary solution:
- exchange data from stdin/stdout.
- make a smaller binding that only wrap few functions (using boost.python, cython or swig)
- use IPC (Inter Process Call) or RPC (Remote Process Call) to exchange data with mr_filter

#### Multiplicity: check performances on the full telescope array

Problem: what improvement can we expect with our cleaning method using the full telescope array (i.e. using multiplicity) ?

Idea: weighted sum of $\Delta\psi_{img}$ (difference between clean and cleaned shower angle of image $img$) per event should give indications on expected improvements. This weighted $\Delta\Psi_{ev}$ sum of event $ev$ is defined as:
$$
\Delta\Psi_{ev} = \sum_{img \in ev} \omega_{img} \Delta\psi_{img}
$$
with $\omega_{img}$ the weight of image $img$ defined as:
$$
\omega_{img} = \frac{\sum_{pix} PE}{\sum_{img}\sum_{pix} PE}
$$

Results on the reconstruction angle accuracy of showers compared to Tailcut (this is results with **cropped** gamma ASTRI images): 
- nearly 60% improvement for the first bin (the shower axis has an angle error < 6°) for faint showers (50 to 100 NPE) versus 40% without multiplicity
- nearly 30% improvements for the first bin (the shower axis has an angle error < 6°) for bright showers (100 to 2000 NPE) versus 5% without multiplicity

We indeed increase the accuracy of the shower angle using multiplicity.

See also https://github.com/jdhp-sap/sap-cta-data-pipeline/blob/master/notebooks/analysis_delta_psi_with_multiplicity.ipynb

#### Proton / Gamma discrimination

Problem: Wavelets provide good reconstruction angle on the images for gamma and for protons.
But do they kill features used for photon/hadron discrimination (i.e. is the proton signature removed by wavelet cleaning) ?

Answer: I made an analysis to have a rough estimation of the impact of the wavelet cleaning on the "discrimination power" of all feature used by Tino. Results are compared to what is observed using Tailcut cleaning.

The conclusion was the following: at this level, the wavelet transform doesn't seems to introduce negative effects for photon/hadron discrimination (compared to Tailcut).

More details: https://dsm-trac.cea.fr/cta/attachment/wiki/Data/sapcta_weekly_meeting_2017_03_23.pdf

## Present results

Present results for ASTRI gamma data (with cropped images) using explicitly reduce the filtering on small scales :
- No energy loss compared to Tailcut (see the 59th slide in http://irfu.cea.fr/Phocea/file.php?class=page&file=595/slides.pdf)
- Wavelets have more images than tailcut with a precise reconstruction angle of showers and less than Tailcut with a bad reconstruction angle (see the 60th slide in http://irfu.cea.fr/Phocea/file.php?class=page&file=595/slides.pdf):
    - slightly more than 40% improvement for the first bin (the shower axis has an angle error < 6°) for faint showers (50 to 100 NPE)
    - nearly 5% improvement for the first bin (the shower axis has an angle error < 6°) for bright showers (100 to 2000 NPE)

More details for ASTRI results (with cropped images): http://irfu.cea.fr/Phocea/file.php?class=page&file=595/slides.pdf

For the photon/hadron discrimination overview study on ASTRI (with cropped images): https://dsm-trac.cea.fr/cta/attachment/wiki/Data/sapcta_weekly_meeting_2017_03_23.pdf

## The prospects for the next months

### Next steps

#### Further tests with GCT instruments

So far, I used a data set containing only proton simulations.
In order to test GCT image cleaning for gamma particle, I will switch to Prod3 simulations.

#### Adapt the framework to any CTA instrument (i.e. to hexagonal grid layouts)

Various ideas to solve this problem:
- patch the wavelet transform code (i.e. in the cosmostat library) to apply a third dimension at the level of the wavelet transform (adding a third loop in the `smooth_bspline` function)
    - it requires to change the image data structure which is a bit tricky. It is too complicated to make this new data structure generic and usable with the full ISAP library but it seems feasible to patch ISAP to let it work only with the few small parts we actually use
    - a much simpler workaround would be to work with a *sparse* cubic data structure
- project the hexagonal grid on a square grid (this is the solution tested by Tino)
- make the cleaning on the 3 possible pair of axis in the hexagonal grid and aggregate the 3 output images
- apply the wavelet transform in a space where the signal has been preprocessed with a Fourier transform (which does not depend on the layout). This solution was suggested by Jean-Luc.

#### Use the time dimension

Most instruments (i.e. all except ASTRI ?) include a time information in the data.
We planed to use this additional dimension within the wavelet transform process.
We expect big improvements using this non integrated signal as it should be very meaningful for the signal/noise and gamma/proton discrimination.

Another advantage of using the time dimension is that some processes at the instrument level (time signal integration, peak detection, ...) won't be necessary anymore and may be removed making the overall data process faster and simpler (i.e. more generic).

#### Use more adapted transformation

Shower images are anisotropic signals.
Using isotropic wavelets has tendencies to make these signals isotropic and thus to suppress information about the orientation of the shower.

Bi-orthogonal wavelets transforms and Curvlet transforms have been tested without success so far.

As suggested by Jean-Luc, more efforts should be made in this direction.

#### Use a better noise model with mf_filter

Optimize the *Generalized Anscombe variance* (see "Astronomical Image and Data Analysis" p.39) to give the correct value to mr_filter's `-m3` option (3 parameters to optimize $\alpha$, $\beta$ $\gamma$ defined as follow: $X = \alpha \sqrt{\beta X + \gamma}$).
Using the right value, the pure noise variance should be equals to 1.

#### Wrapper/binding

Use the *mr_filter* binding when it will be ready (autumn 2017).

### Ideas for further steps

- Use better features for the gamma/proton classifier (e.g. use directly second order moments of the Wavelet coefficients distribution as an input of the classifier in addition to the Hillas parameters)
- Use automatic feature extraction methods (?)

- Use better machine learning methods for the gamma/proton classifier (e.g. SVM)
- Test deep learning on the image space. Cons:
    - it won't use any cosmostat tool anymore and thus causes political issues...
    - require a lot of (developing and material) resource

- Assess images using the actual angle of the monte carlo shower instead of using the angle obtain with Hillas parametrization on the clean image (suggested by Jean-Luc)
- Use wavelets only to detect the signal position and give this information to a *matched filter* (to reduce the number of dimension of the search) which will be in charge of the actual cleaning of the signal (suggested by Jean-Luc and Sandrine)

- Add an option in mr_filter to use a tailored noise distribution (the actual noise distribution of our problem) 
- As the signal is very local, use a 2 pass wavelet filtering
    - the first pass would detect the signal position
    - the second pass would locally adjust threshold (reduce thresholds around the signal detected by the first pass)
- Learn the "optimal" mother wavelet from a set of pure signal

Update the execution time measurements and optimize it:
- Compile with better optimization flags: easy Use efficient third parties libraries
- Shared memory parallelization (OpenMP) Distributed memory parallelization (MPI, ...)
- Customize the algorithms implementation
- Check some other libraries
- Check some other algorithms ...

Charge loss:
as an alternative to mr_filter's option `-K` (remove the last plane), we may use the first 3 planes (among the 4 used) to make a mask indicating which pixels are kept (i.e. pixels being above the filtering threshold in any of these 3 planes) and use this mask to keep only these pixels in the last plane.